In [1]:

import os, shutil
import tensorflow.keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import array_to_img

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom
from PIL import Image
import cv2
import shutil

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
#get working directory
os.getcwd()

'C:\\Users\\Mojtaba Jafari Tadi\\Projects_AMK\\Radiology Project\\fatpad'

In [4]:
os.listdir()

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'brucechou1983_CheXNet_Keras_0.3.0_weights.h5',
 'CNN_automated_medical_diagnostics_five_layers_chopped_two_classes_trained_on_twenty_epochs_Adam_0.0001.ipynb',
 'CNN_FATPAD_SimplePlainNN.ipynb',
 'CNN_FATPAD_TransferLearning-DenseNet.ipynb',
 'CNN_FATPAD_TransferLearning-EfficientNet.ipynb',
 'CNN_FATPAD_TransferLearning-NASNet.ipynb',
 'CNN_FATPAD_TransferLearning.ipynb',
 'Labels_FatPadDetection.xlsx',
 'Prepare_Xray_Data.ipynb',
 'README.md']

In [5]:
#change working directory to where the datasets are
root_directory=os.chdir('C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project')

In [6]:
#Provide train, validation, and test to tensorflow-keras ImageDataGenerator
base_dir_train = 'C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_Split/train'
base_dir_validate = 'C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_Split/validation'
base_dir_test = 'C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_Split/test'

print(base_dir_train)

C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_Split/train


In [17]:
#run tensorflow-keras ImageDataGenerator on the directory which contains train, validate and test
#train_datagen = ImageDataGenerator(
 #   rescale=1./255, # new parameter (maps pixel values to floats)
 #   rotation_range=40,
 #   width_shift_range=0.2,
  ##  height_shift_range=0.2,
  #  shear_range=0.2,
  #  zoom_range=0.25,
  #  horizontal_flip=True,
  #  fill_mode='nearest') # Note: I added this, too

train_datagen = ImageDataGenerator(samplewise_center=False, 
                              samplewise_std_normalization=False, 
                              horizontal_flip = True, 
                              vertical_flip = False, 
                              height_shift_range= 0.15, 
                              width_shift_range=0.15, 
                              rotation_range=5, 
                              shear_range = 0.1,
                              fill_mode = 'reflect',
                              zoom_range=0.15)

# The validation data should not be augmented (just rescaled)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        base_dir_train,
        color_mode='rgb',
        target_size=(224, 224), 
        batch_size=16, 
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        base_dir_validate, 
        color_mode='rgb',
        target_size=(224, 224), 
        batch_size=16, 
        class_mode='binary')

Found 3519 images belonging to 2 classes.
Found 1005 images belonging to 2 classes.


In [18]:
import matplotlib.pyplot as plt


def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()


In [19]:
train_generator.image_shape


(224, 224, 3)

In [23]:
from tensorflow.keras.applications import NASNetMobile
inputs = layers.Input(shape=train_generator.image_shape)

root_model = NASNetMobile(include_top=False, input_tensor=inputs, weights='imagenet')

root_model = tensorflow.keras.Model(inputs=root_model.input, outputs=root_model.layers[-5].output)

# Freeze the pretrained weights
root_model.trainable = False

# Rebuild top
model = models.Sequential()
model.add(root_model)

model.add(layers.GlobalAveragePooling2D(name="avg_pool"))
model.add(layers.BatchNormalization())

top_dropout_rate = 0.2
model.add(layers.Dropout(top_dropout_rate, name="top_dropout"))

model.add(layers.Dense(1, activation="sigmoid", name="pred"))

# Compile
#optimizer = tensorflow.keras.optimizers.Adam(learning_rate=1e-2)
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.001), metrics=["accuracy"])
#model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
functional_13 (Functional)   (None, 7, 7, 176)         3925812   
_________________________________________________________________
avg_pool (GlobalAveragePooli (None, 176)               0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 176)               704       
_________________________________________________________________
top_dropout (Dropout)        (None, 176)               0         
_________________________________________________________________
pred (Dense)                 (None, 1)                 177       
Total params: 3,926,693
Trainable params: 529
Non-trainable params: 3,926,164
_________________________________________________________________


In [24]:

hist = model.fit(train_generator,
      epochs=50,
      validation_data=validation_generator,
      verbose=1)

plot_hist(hist)


Epoch 1/50
 31/220 [===>..........................] - ETA: 53s - loss: 0.7560 - accuracy: 0.5020

KeyboardInterrupt: 

In [ ]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=20)
test_acc